In [6]:
import sys
import os
parent_dir = os.path.abspath('..')
sys.path.append(parent_dir)
import system.robot as robotSystem
import system.utils.dynamic_properties as part_properties
import numpy as np
import casadi as ca

In [7]:
alpha = robotSystem.RobotDynamics()
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
path_to_urdf = os.path.join(
    project_root,
    'usage',
    'urdf','reach_alpha_5',
    'alpha_5_robot.urdf'
)
alpha.from_file(path_to_urdf)

root = "base_link"
tip = "alpha_standard_jaws_base_link"

joint_min = np.array([1.00, 0.01, 0.01, 0.01])
joint_max = np.array([5.50, 3.40, 3.40, 5.70])
base_T0 = [0.190, 0.000, -0.120, 3.142, 0.000, 0.000] #floating base mount
base_T1 = [0.0, 0.000, 0.0, 0.0, 0.000, 0.000] #fixed base mount
kinematic_dict, K, P, L, D, C, g, qdd, joint_torque = alpha.build_model(root, tip, floating_base=True)
inertial_origins_params, m_params, I_params, vec_g, q, q_dot, q_dotdot, tau, base_pose, world_pose = kinematic_dict['parameters']

In [8]:
L.size(), K.size(), P.size(), D.size(), g.size()

((1, 1), (1, 1), (1, 1), (4, 4), (4, 1))

In [9]:
D_f = ca.Function('inertia_matrix', [ca.vertcat(*inertial_origins_params), 
                               ca.vertcat(*m_params),
                               ca.vertcat(*I_params),
                               vec_g,
                               q,
                               base_pose,
                               world_pose ],[D , g])

In [10]:
for i in range(10000):
    joint_min = np.array([1.00, 0.01, 0.01, 0.01])
    joint_max = np.array([5.50, 3.40, 3.40, 5.70])
    q_sample = np.random.uniform(joint_min, joint_max)

    robot_inertia_matrix, gravity_vector = D_f([5e-3 ,-1e-3 ,16e-3,
        5e-3, -1e-3, 16e-3,
        5e-3, -1e-3, 16e-3,
        5e-3, -1e-3, 16e-3],
        [0.341, 0.341, 0.341, 0.341],
        [1e-2, 1e-2, 1e-2, 0, 0, 0, 1e-2, 1e-2, 1e-2, 0, 0, 0, 1e-2, 1e-2, 1e-2, 0, 0, 0, 1e-2, 1e-2, 1e-2, 0, 0, 0],
        [0, 0, -9.81],
        q_sample,
        [0.190, 0.000, -0.120, 3.142, 0.000, 0.000],
        [0.0, 1.0, 0, 0, 0, 0])
    spd_state = part_properties.is_spd(robot_inertia_matrix)
    if not spd_state:
        raise ValueError("not spd")
